# Iterables, Iterators, and Generators
The yield keyword was added in Python 2.2 (2001). The yield keyword allows the construction of generators, which work as iterators.

## Sentence Take #1: A Sequence of Words

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)


In [2]:
s = Sentence('"The time has come," the Walrus said,') 

In [3]:
s

Sentence('"The time ha... Walrus said,')

In [4]:
for word in s:
    print(word)


The
time
has
come
the
Walrus
said


In [6]:
s[0], s[1], s[2]

('The', 'time', 'has')

## Why Sequences Are Iterable: The iter Function
Whenever the interpreter needs to iterate over an object x, it automatically calls iter(x). The iter built-in function:
-  Checks whether the object implements __iter__, and calls that to obtain an iterator.
-  If __iter__ is not implemented, but __getitem__ is implemented, Python creates an iterator that attempts to fetch items in order, starting from index 0 (zero).
-  If that fails, Python raises TypeError, usually saying “C object is not iterable,” where C is the class of the target object.

That is why any Python sequence is iterable: they all implement __getitem__. In fact, the standard sequences also implement __iter__, and yours should too, because the special handling of __getitem__ exists for backward compatibility reasons and may be
gone in the future (although it is not deprecated as I write this).

## Iterables Versus Iterators